In [ ]:
import pandas as pd
import os
os.chdir('../')
from kernel import utils, data_processing, modelling

# 1. Load Training Data

In [ ]:
df = (pd.read_csv('./data/clf_train.csv').pipe(utils.regularize_columns)
                                     .pipe(utils.detect_id_col)
     )

In [ ]:
df_fea, labels = utils.split_feature_target(df, 'survived')
df_fea.drop(['ticket', 'name'], axis=1, inplace=True)

In [ ]:
dp = data_processing.DataProcessor(df_fea, 'titanic', True)
df_inf = pd.read_csv('./data/clf_test.csv').pipe(utils.regularize_columns)

In [ ]:
X, y = dp.data.values, labels.values

In [ ]:
clf, params, est = modelling.train_lr_classifier(X, y)

In [ ]:
model = est(solver='lbfgs', **params)
model.fit(X, y)

In [ ]:
df_inf['Survived'] = model.predict(dp.transform(df_inf).values)

In [ ]:
df_inf = df_inf.reset_index().rename(columns={'passengerid': 'PassengerId'})

In [ ]:
df_inf[['PassengerId', 'Survived']].to_csv('./data/inf.csv', index=False)